In [1]:
import numpy as np
import os
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ProcessPoolExecutor

import sys
script_path = os.path.abspath("pyscripts")
if script_path not in sys.path:
    sys.path.append(script_path)
from utils import *

In [2]:
folderpath =  '/amethyst/s0/fbx5002/geolab_storage_V3/data/IoT_Temp/WeatherUndergound_NYC_stationfiles_2015-2020/'
weather_files = list(map(lambda f: folderpath + f, os.listdir(folderpath)))
chunk_size = 20
files_chunk = chunk_files(weather_files, chunk_size)
print(len(weather_files)); files_chunk[1][:2]

130


['/amethyst/s0/fbx5002/geolab_storage_V3/data/IoT_Temp/WeatherUndergound_NYC_stationfiles_2015-2020/KNJJERSE108.csv',
 '/amethyst/s0/fbx5002/geolab_storage_V3/data/IoT_Temp/WeatherUndergound_NYC_stationfiles_2015-2020/KNJJERSE118.csv']

In [3]:
weather_ground = {}
with ProcessPoolExecutor(max_workers=4) as executor:
    for ret in executor.map(read_weather_underground_files_2015_2020, files_chunk):
        weather_ground.update(ret)
len(weather_ground.keys())

130

In [4]:
# start: 2014-12-31 00:00:00  end: 2020-09-13 23:00:00
print(all(list(map(lambda x: x['local_datetime'].iloc[0]=='2014-12-31 00:00:00' and
         x['local_datetime'].iloc[-1]=='2020-09-13 23:00:00', weather_ground.values()))))

start = datetime.strptime('2014-12-31 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2020-09-13 23:00:00', '%Y-%m-%d %H:%M:%S')
diff = end-start
hours = diff.days*24 + diff.seconds//3600  # how many hours should be there

weather_time_2015_2020 = [start+ timedelta(hours=1*i) for i in range(hours+1)]
print(weather_time_2015_2020[0], weather_time_2015_2020[-1])
np.save('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/weather_time_2015_2020.npy', weather_time_2015_2020)

True
2014-12-31 00:00:00 2020-09-13 23:00:00


In [5]:
missing = continous_breakpoint(weather_ground['KCTSTAMF84']['local_datetime'], time_format ='%Y-%m-%d %H:%M:%S')
len(weather_time_2015_2020) == weather_ground['KCTSTAMF84'].shape[0] + sum([x[1] for x in missing])

True


In [7]:
weather_station_2015_2020 = []
missingratio_all = {}
for station, data in weather_ground.items():
    weather_station_2015_2020.append([station, data['lon'][0], data['lat'][0]])
    missing = continous_breakpoint(data['local_datetime'], time_format ='%Y-%m-%d %H:%M:%S')
    missingratio_all[station] = sum([x[1] for x in missing])/len(weather_time_2015_2020)

min_ratio = missingratio_all[min(missingratio_all, key=lambda x: missingratio_all[x])]
max_ratio = missingratio_all[max(missingratio_all, key=lambda x: missingratio_all[x])]
print(min_ratio, max_ratio)

weather_station_2015_2020 = pd.DataFrame(weather_station_2015_2020, columns = ['station', 'lon', 'lat'])
weather_station_2015_2020.to_csv("/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/weather_station_2015_2020.csv", index=False)

0.0007597568777991043 0.0007597568777991043


In [7]:
infos = {}
for station, data in weather_ground.items():
    tmp = data['local_datetime']
    missing = continous_breakpoint(tmp, time_format ='%Y-%m-%d %H:%M:%S')
    # change temperature to Celsius
    data['temperature'] = (data['temperature'] -32)/1.8
    infos[station] = fill_na_breakpoint(np.array(data['temperature']), missing)

In [8]:
all([len(x)==len(weather_time_2015_2020) for x in infos.values()])
all(list(infos.keys()) == weather_station_2015_2020['station'])

True

In [9]:
weather_obs_2015_2020 = np.array(list(infos.values()))
np.save('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/weather_obs_2015_2020.npy', weather_obs_2015_2020)